In [36]:
import os
import json
import logging
import operator
from typing import (
    Annotated, List, Dict, TypedDict, Callable, Union, Literal, Optional
)

from pydantic import (
    BaseModel,
    Field
)
from litellm import completion
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

from IPython.display import Image, display

# Model

In [15]:
path_to_openai_key:str = os.path.expanduser('~/.openai/api_key')
with open(path_to_openai_key, 'r', encoding='utf-8') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()

MODEL: str = "openai/gpt-4o"

## Completion Model

LiteLLM uses Open AI input format as the standardised schema.

* [completion - Input Params](https://docs.litellm.ai/docs/completion/input)

> LiteLLM accepts and translates the OpenAI Chat Completion params across all providers.

See the OpenAI document for the available options.

* [OpenAI API - Chat Completion](https://platform.openai.com/docs/api-reference/chat/create)



### Prompt

In [107]:
system_message = """
You are a helpful assistant to help autonomous task-execution agents.

PRIMARY ROLE
- Transform user queires into explicit, executable plans or tool calls.
- Be meticulously step by step to derive the plans or tool calls.
- Execute only steps that are logically justified and safe.
- Prefer correctness and completeness over speed.
- Think deeper and validate the thought processes taken.

OPERATING RULES
1. Never assume missing information.
2. If required information is missing, explicitly ask for it.
3. Do not invent facts, APIs, files, or system state.
4. If a task exceeds your authority or tools, stop and report.
5. If the query is unethical or can violate regulations or laws, stop and report.

REASONING
- Break tasks into explicit substeps.
- Validate each step before proceeding.
- Track assumptions explicitly.
- Detect contradictions and stop if found.|

TOOL USAGE
- Use tools when you cannot handle by yourself.
- Do not call tools speculatively.
- Validate tool outputs before using them downstream.

OUTPUT CONTRACT
- Produce outputs that are:
  - deterministic
  - reproducible
  - minimal but sufficient
- Clearly separate:
  - conclusions
  - assumptions
  - open questions

FAILURE MODE
- When uncertain, return:
  "Cannot proceed safely due to missing or conflicting information."
- Never guess.
"""

user_message:str = "What is the QF26 flight status today?"

## Model Query without Tools

LLM does not know how to get NYC time.

In [109]:
response = completion(
    model=MODEL,
    messages=[
      {
        "role": "system",
        "content": system_message
      },
      {
        "role": "user",
        "content": user_message
      }
    ],
    temperature=1.0,
    max_tokens=120,
    tool_choice=None,
)

/Users/oonisim/venv/ml/lib/python3.10/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected 10 fields but got 6 for type `Message` with value `Message(content="I cannot provide real-time inform...pecific_fields={'refusal': None}, annotations=[])` - serialized value may not be as expected.
  PydanticSerializationUnexpectedValue: Expected `StreamingChoices` but got `Choices` with value `Choices(finish_reason='st...ider_specific_fields={})` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [110]:
print(json.dumps(response.model_dump(), indent=2))

{
  "id": "chatcmpl-Cq8UHSj9ujxDvE7hOvgVwENaUT6FF",
  "created": 1766541717,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_deacdd5f6f",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I cannot provide real-time information such as flight status. To find the status of QF26, you can check a reliable flight tracking website or the airline's official sources. If you need guidance on how to do this, feel free to ask!",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "provider_specific_fields": {
          "refusal": null
        },
        "annotations": []
      },
      "provider_specific_fields": {}
    }
  ],
  "usage": {
    "completion_tokens": 50,
    "prompt_tokens": 300,
    "total_tokens": 350,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "re

---
# Tools

## Defining Tool Schema

Suppose using [Tavily - Search](https://docs.tavily.com/sdk/python/reference#tavily-search) as a general search tool such as getting time or weather of a city. We need to tell LLM of the schema that Tavily Search accepts. Find the available parameters from the API document or using [Tavily Search Playground](https://app.tavily.com/home). Then LiteLLM can get the Tool Schema from SearchTool.model_json_schema()

In [111]:
class SearchTool(BaseModel):
    # DocString tells LLM what this tool is for.
    """Search the web for general topics such as time, news, weather, events."""
    query: str = Field(description="The search query to look up")

    #--------------------------------------------------------------------------------
    # Attributes of Tavily Search API (https://docs.tavily.com/sdk/python/reference).
    # LLM can genrate the arguments based on these information.
    #--------------------------------------------------------------------------------
    topic: Literal["general", "news", "finance"] = Field(
        default="general",
        description="Category of search. Use 'news' for current events/politics, 'finance' for market data."
    )
    
    search_depth: Literal["basic", "advanced"] = Field(
        default="basic",
        description="Use 'basic' for quick facts. Use 'advanced' for complex queries needing more context."
    )
    
    time_range: Optional[Literal["day", "week", "month", "year"]] = Field(
        default=None,
        description="Filter results by publication date. Especially useful with topic='news'."
    )
    
    max_results: int = Field(
        default=5, ge=1, le=10,
        description="Number of search results to return."
    )

In [112]:
print(json.dumps(SearchTool.model_json_schema(), indent=2))

{
  "description": "Search the web for general topics such as time, news, weather, events.",
  "properties": {
    "query": {
      "description": "The search query to look up",
      "title": "Query",
      "type": "string"
    },
    "topic": {
      "default": "general",
      "description": "Category of search. Use 'news' for current events/politics, 'finance' for market data.",
      "enum": [
        "general",
        "news",
        "finance"
      ],
      "title": "Topic",
      "type": "string"
    },
    "search_depth": {
      "default": "basic",
      "description": "Use 'basic' for quick facts. Use 'advanced' for complex queries needing more context.",
      "enum": [
        "basic",
        "advanced"
      ],
      "title": "Search Depth",
      "type": "string"
    },
    "time_range": {
      "anyOf": [
        {
          "enum": [
            "day",
            "week",
            "month",
            "year"
          ],
          "type": "string"
        },
     

## Model Query with Tools

In [113]:
path_to_tavily_key:str = os.path.expanduser('~/.tavily/api_key')
with open(path_to_tavily_key, 'r', encoding='utf-8') as file:
    os.environ["TAVILY_API_KEY"] = file.read().strip()

In [114]:
response = completion(
    model=MODEL,
    messages=[
      {
        "role": "system",
        "content": system_message
      },
      {
        "role": "user",
        "content": user_message
      }
    ],
    temperature=1.0,
    max_tokens=120,
    # --------------------------------------------------------------------------------
    # Provide Tools
    # --------------------------------------------------------------------------------
    tool_choice="auto",
    tools = [{
        "type": "function",
        "function": {
            "name": SearchTool.__name__,
            "description": SearchTool.__doc__,
            "parameters": SearchTool.model_json_schema()
        }
    }]
)

/Users/oonisim/venv/ml/lib/python3.10/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected 10 fields but got 6 for type `Message` with value `Message(content='To provide the status of QF26 fli...pecific_fields={'refusal': None}, annotations=[])` - serialized value may not be as expected.
  PydanticSerializationUnexpectedValue: Expected `StreamingChoices` but got `Choices` with value `Choices(finish_reason='to...ider_specific_fields={})` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [115]:
print(json.dumps(response.model_dump(), indent=2))

{
  "id": "chatcmpl-Cq8UJhVCTyoPjVxQJjCWORDVtRrvP",
  "created": 1766541719,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_deacdd5f6f",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "message": {
        "content": "To provide the status of QF26 flight today, I need to perform a search. I will proceed with searching for the current status of this flight.",
        "role": "assistant",
        "tool_calls": [
          {
            "function": {
              "arguments": "{\"query\":\"QF26 flight status today\",\"topic\":\"general\",\"search_depth\":\"basic\"}",
              "name": "SearchTool"
            },
            "id": "call_G5Iv5254qonptStZOaVqul4D",
            "type": "function"
          }
        ],
        "function_call": null,
        "provider_specific_fields": {
          "refusal": null
        },
        "annotations": []
      },
      "provider_specific_fields": {}
    }
  ],
  "usa

---

# LangGraph

In [53]:
def call_tool(state: AgentState):
    last_msg = state.messages[-1]
    tool_call = last_msg["tool_calls"][0]
    
    # Extract arguments - these now include topic, search_depth, etc.
    args = json.loads(tool_call["function"]["arguments"])
    
    # 5. DYNAMIC TAVILY CALL
    # Passing **args allows Tavily to receive topic, time_range, etc. automatically
    search_data = tavily.search(**args)
    
    content = "\n".join([
        f"Source: {r['title']}\nURL: {r['url']}\nContent: {r['content']}\n---" 
        for r in search_data['results']
    ])
    
    return {"messages": [{
        "role": "tool",
        "tool_call_id": tool_call["id"],
        "name": "SearchTool",
        "content": content
    }]}

# 6. ROUTER & GRAPH
def router(state: AgentState) -> Literal["call_tool", "__end__"]:
    last_msg = state.messages[-1]
    return "call_tool" if last_msg.get("tool_calls") else "__end__"

builder = StateGraph(AgentState)
builder.add_node("llm", call_model)
builder.add_node("call_tool", call_tool)
builder.add_edge(START, "llm")
builder.add_conditional_edges("llm", router)
builder.add_edge("call_tool", "llm")
app = builder.compile()

# 7. EXECUTION
# Example: Try asking for "Latest news about Bitcoin today" 
# to see the LLM choose topic="news" and time_range="day".
inputs = {"messages": [{"role": "user", "content": "What are the top news headlines in AI from the last week?"}]}

for event in app.stream(inputs, stream_mode="values"):
    last_msg = event["messages"][-1]
    role = last_msg["role"].upper()
    
    # Display logic to show tool parameters used
    if "tool_calls" in last_msg:
        params = json.loads(last_msg["tool_calls"][0]["function"]["arguments"])
        print(f"\n--- {role} (Calling Search with params: {params}) ---")
    else:
        print(f"\n--- {role} ---")
        print(last_msg.get("content") or "Processing...")

NameError: name 'AgentState' is not defined

In [ ]:
# 7. EXECUTION
inputs = {"messages": [{"role": "user", "content": "What are the top news headlines in AI from the last week?"}]}

for event in app.stream(inputs, stream_mode="values"):
    last_msg = event["messages"][-1]
    
    # Check if it's a dictionary (from our nodes) or a LiteLLM object
    role = last_msg.get("role", "assistant").upper() if isinstance(last_msg, dict) else last_msg.role.upper()
    
    # Robust display logic
    tool_calls = last_msg.get("tool_calls") if isinstance(last_msg, dict) else getattr(last_msg, "tool_calls", None)

    if tool_calls:
        # tool_calls[0] is an object, but function.arguments is a string
        first_call = tool_calls[0]
        # Handle both object-style and dict-style access
        args_str = first_call["function"]["arguments"] if isinstance(first_call, dict) else first_call.function.arguments
        params = json.loads(args_str)
        print(f"\n--- {role} (Calling Search with params: {params}) ---")
    else:
        content = last_msg.get("content") if isinstance(last_msg, dict) else getattr(last_msg, "content", "")
        print(f"\n--- {role} ---")
        if content:
            print(content)
        else:
            print("Processing...")